In [1]:
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api.formatters import TextFormatter

In [5]:
# proxies = {'https': 'https://crab233.cloudns.biz/proxy/'}
transcript_list = YouTubeTranscriptApi.list_transcripts('D_TyrPyGXyM&t=624s')
print(transcript_list)

TranscriptsDisabled: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=D_TyrPyGXyM&t=624s! This is most likely caused by:

Subtitles are disabled for this video

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Please add which version of youtube_transcript_api you are using and provide the information needed to replicate the error. Also make sure that there are no open issues which already describe your problem!

In [6]:
# trz7g-wilxs = "5 RULES FOR THE REST OF YOUR LIFE" | Matthew McConaughey MOTIVATIONAL SPEECH
# UF9Iqmg94tk = "Consistent Hashing | Algorithms You Should Know #1"
transcript = YouTubeTranscriptApi.get_transcript('UF9Iqmg94tk', languages=['en-US'])
# transcript = YouTubeTranscriptApi.get_transcript('XPMysVisyWE')
formatter = TextFormatter()
text_formatted = formatter.format_transcript(transcript)
with open('transcript.txt', 'w', encoding='utf-8') as text_file:
    text_file.write(text_formatted)

In [6]:
%cat transcript.txt

What do DynamoDB, Apache Cassandra, Discord, and 
Akamai CDN have in common? They all use consistent  
hashing. Now, what is consistent hashing? Why do all 
the cool kids use it? In this video, we'll learn  
all about it. Let's dive right in. In a large scale 
distributed system, data does not fit on a single  
server. They are distributed across many machines. 
This is called horizontal scaling. To build such  
a system with predictable performance. it is 
important to distribute the data evenly across  
those servers. A common method to distribute 
data as evenly as possible among servers is  
simple hashing. This is how it works. First, for each 
object, we hash its key with a hashing function  
like MD5 or MurmurHash. This maps the object 
key to a known range of numerical values.  
A good hashing function distributes the 
hashes evenly across the entire range.  
Second, we perform the modulo operation on the 
hash against the number of servers. This determines  
which servers the 

In [7]:
from langchain.document_loaders import TextLoader

loader = TextLoader("./transcript.txt")
docs = loader.load()

In [8]:
print(f'You have {len(docs)} document(s) in your data')
print(f'There are {len(docs[0].page_content)} characters in your document')

You have 1 document(s) in your data
There are 6768 characters in your document


In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
split_docs = text_splitter.split_documents(docs)

In [10]:
print(f'You have {len(split_docs)} split document(s)')


You have 7 split document(s)


In [11]:
from dotenv import load_dotenv
import os
from langchain_community.llms.cloudflare_workersai import CloudflareWorkersAI
from langchain_community.llms.tongyi import Tongyi
from langchain_openai import ChatOpenAI

load_dotenv(override=True)

account_id = os.getenv('CF_ACCOUNT_ID')
api_token = os.getenv('CF_API_TOKEN')
print(account_id)
print(api_token)

# CloudflareWorkersAI
model = '@cf/meta/llama-3-8b-instruct'
cf_llm = CloudflareWorkersAI(
    account_id=account_id,
    api_token=api_token,
    model=model
)

DASHSCOPE_API_KEY = os.getenv('DASHSCOPE_API_KEY')
print(DASHSCOPE_API_KEY)

# qwen
qw_llm = Tongyi(
    model='qwen2-1.5b-instruct'
)

# qwen 兼容 openai的接口
qw_llm_openai = ChatOpenAI(
    openai_api_base='https://dashscope.aliyuncs.com/compatible-mode/v1',
    openai_api_key=DASHSCOPE_API_KEY,
    model_name="qwen2-1.5b-instruct",
    temperature=0,
    streaming=True,
    verbose=True,
)

api_key = os.getenv('OPENAI_API_KEY')
base_url = os.getenv('OPENAI_API_BASE')
print(api_key)
print(base_url)

# openai/moonshot
ms_llm = ChatOpenAI(
    openai_api_base=base_url,
    openai_api_key=api_key,
    model_name="moonshot-v1-8k",
    temperature=0.7,
)

8483c3ec7a0cbc54a8d660b5b9002b04
Gcllof8ze6dgtcqFI5FQZ2SD_5tfCD4Db7NuS6jn
sk-01c5003340c3453b934052d737d45e01
sk-UGVpjuTwo2Q8pewoqUDfckw1A0pbSDli9ElFMeS9WareKknG
https://api.moonshot.cn/v1/


In [12]:
from langchain.chains.summarize import load_summarize_chain

chain = load_summarize_chain(qw_llm_openai, chain_type="map_reduce", verbose=True)

In [13]:
input_docs = split_docs[:2]

In [14]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    chain.invoke(input_docs)
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Successful Requests: {cb.successful_requests}")
    print(f"Total Cost (USD): ${cb.total_cost}")



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"What do DynamoDB, Apache Cassandra, Discord, and 
Akamai CDN have in common? They all use consistent  
hashing. Now, what is consistent hashing? Why do all 
the cool kids use it? In this video, we'll learn  
all about it. Let's dive right in. In a large scale 
distributed system, data does not fit on a single  
server. They are distributed across many machines. 
This is called horizontal scaling. To build such  
a system with predictable performance. it is 
important to distribute the data evenly across  
those servers. A common method to distribute 
data as evenly as possible among servers is  
simple hashing. This is how it works. First, for each 
object, we hash its key with a hashing function  
like MD5 or MurmurHash. This maps the object 
key to a known range of numerical values.  
A good hashing function distributes the 
hashes